In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d zarajamshaid/language-identification-datasst

Dataset URL: https://www.kaggle.com/datasets/zarajamshaid/language-identification-datasst
License(s): DbCL-1.0
  0% 0.00/5.53M [00:00<?, ?B/s]
100% 5.53M/5.53M [00:00<00:00, 120MB/s]


In [4]:
from zipfile import ZipFile
dataset = '/content/language-identification-datasst.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

In [6]:
dataset=pd.read_csv("/content/dataset.csv")

In [7]:
dataset.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [8]:
dataset.tail()

,Text,language
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese
21999,aprilie sonda spațială messenger a nasa și-a ...,Romanian


In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
dataset['language']=le.fit_transform(dataset['language'])

In [10]:
dataset.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,4
1,sebes joseph pereira thomas på eng the jesuit...,17
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,19
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,18
4,de spons behoort tot het geslacht haliclona en...,2


In [11]:
dataset['language'].value_counts()

,count
language,
4,1000
17,1000
19,1000
18,1000
2,1000
8,1000
20,1000
10,1000
21,1000


In [12]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(dataset,test_size=0.2)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words=20000,oov_token="Krish")
tokenizer.fit_on_texts(train_data['Text'])
sequences=tokenizer.texts_to_sequences(train_data['Text'])
sequences1=tokenizer.texts_to_sequences(test_data['Text'])
X_train=pad_sequences(sequences,maxlen=100)
X_test=pad_sequences(sequences1,maxlen=100)

In [15]:
X_train

array([[   0,    0,    0, ...,  463,   12,  372],
       [   0,    0,    0, ...,    1,    1,    1],
       [   0,    0,    0, ...,   40, 1038,    1],
       ...,
       [   0,    0,    0, ...,    1,    1,    1],
       [   0,    0,    0, ...,    1,    1,    1],
       [ 314,   14,    9, ...,    1,  611,    1]], dtype=int32)

In [16]:
X_test

array([[    0,     0,     0, ...,    12,     5,  2091],
       [  128,   107,   452, ...,  2671,     1, 12190],
       [    0,     0,     0, ...,     1,     1,  3065],
       ...,
       [    0,     0,     0, ...,  4774,  3204,     1],
       [    0,     0,     0, ...,   774,  2696,     1],
       [    0,     0,     0, ..., 14107,     1,     1]], dtype=int32)

In [17]:
y_train=train_data['language']
y_test=test_data['language']

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [19]:
model=Sequential()
model.add(Embedding(input_dim=20000,output_dim=128,input_length=100,input_shape=(100,)))
model.add(LSTM(units=128,activation='tanh',dropout=0.2))
model.add(Dense(units=22,activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 22)                2838      
                                                                 
Total params: 2694422 (10.28 MB)
Trainable params: 2694422 (10.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [22]:
model.fit(X_train,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
440/440 [==============================] - 77s 171ms/step - loss: 1.3759 - accuracy: 0.5651 - val_loss: 0.4822 - val_accuracy: 0.8389
Epoch 2/10
440/440 [==============================] - 74s 168ms/step - loss: 0.3757 - accuracy: 0.8785 - val_loss: 0.2332 - val_accuracy: 0.9227
Epoch 3/10
440/440 [==============================] - 74s 168ms/step - loss: 0.2129 - accuracy: 0.9210 - val_loss: 0.2229 - val_accuracy: 0.9213
Epoch 4/10
440/440 [==============================] - 74s 167ms/step - loss: 0.1968 - accuracy: 0.9245 - val_loss: 0.1980 - val_accuracy: 0.9256
Epoch 5/10
440/440 [==============================] - 74s 168ms/step - loss: 0.1369 - accuracy: 0.9389 - val_loss: 0.1922 - val_accuracy: 0.9327
Epoch 6/10
440/440 [==============================] - 74s 168ms/step - loss: 0.1349 - accuracy: 0.9397 - val_loss: 0.2122 - val_accuracy: 0.9216
Epoch 7/10
440/440 [==============================] - 73s 166ms/step - loss: 0.1076 - accuracy: 0.9472 - val_loss: 0.1873 - val_ac

In [23]:
loss,accuracy=model.evaluate(X_test,y_test)

138/138 [==============================] - 5s 34ms/step - loss: 0.2227 - accuracy: 0.9211


In [24]:
loss

0.22272765636444092

In [25]:
accuracy

0.9211363792419434

In [29]:
def prediction(text):
    sequence=tokenizer.texts_to_sequences([text])
    padded_sequence=pad_sequences(sequence,maxlen=100)
    sentiment=model.predict(padded_sequence)
    sentiment=np.argmax(sentiment)
    return [sentiment]

In [34]:
sen="kemunculan pertamanya adalah ketika mencium kakak kelasnya kyoko sejak yuuki meminta agar sakura mer..."
le.inverse_transform(prediction(sen))

1/1 [==============================] - 0s 31ms/step


array(['Latin'], dtype=object)